In [152]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

False

In [153]:
import os
from langchain.graphs import Neo4jGraph

graph = Neo4jGraph(
    url="bolt://localhost:7687",
    username="neo4j",
    password="neo4j"
)

KeyboardInterrupt: 

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = ChatOllama(model="mistral", temperature=0)
model= OllamaFunctions(model="mistral", verbose=True)

In [ ]:
# loading the documents
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./input/', glob="**/*.txt", use_multithreading=True, show_progress=True)
docs = loader.load()
len(docs)

100%|██████████| 3/3 [00:00<00:00, 30.35it/s]


3

In [ ]:
# splitting the documents
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=200,
  length_function=len,
  is_separator_regex=False
)
split_docs = splitter.split_documents(docs)
len(split_docs)

12

In [ ]:
# setup pydantic classes
from langchain.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship
)
from langchain.pydantic_v1 import Field, BaseModel
from typing import List, Optional

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")


class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")


class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(..., description="List of relationships in the knowledge graph")

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_core.prompts import HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
  sys_prompt = SystemMessage(content=f"""# Knowledge Graph Instructions for GPT-4
  ## 1. Overview
  You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
  - **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
  - The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
  ## 2. Labeling Nodes
  - **Consistency**: Ensure you use basic or elementary types for node labels.
    - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
  - **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
  {'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
  {'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
  ## 3. Handling Numerical Data and Dates
  - Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
  - **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
  - **Property Format**: Properties must be in a key-value format.
  - **Quotation Marks**: Never use escaped single or double quotes within property values.
  - **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
  ## 4. Coreference Resolution
  - **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
  If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"), 
  always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.  
  Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial. 
  ## 5. Strict Compliance
  Adhere to the rules strictly. Non-compliance will result in termination.""")

  prompt = ChatPromptTemplate.from_messages([
    sys_prompt,
    SystemMessagePromptTemplate.from_template(
        "Output format instructions: {format_instructions}"),
    HumanMessagePromptTemplate.from_template("Use the given format to extract information from the following input: {input}"),
    HumanMessage(content="Tip: Make sure to answer in the correct format"),
  ])
  from langchain.output_parsers import PydanticOutputParser
  parser = PydanticOutputParser(pydantic_object=KnowledgeGraph)
  partial_prompt = prompt.partial(
      format_instructions=parser.get_format_instructions())
  # use LCEL to pass the prompt to the model and force a structured response
  extraction_chain = partial_prompt | llm 

  return extraction_chain

In [ ]:
from langchain.graphs.graph_document import GraphDocument
from langchain_core.documents import Document


def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

def extract_and_store_graph(
        document: Document,
        nodes: Optional[List[str]] = None,
        rels: Optional[List[str]] = None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.invoke({"input": document.page_content})
    # for chunk in extract_chain.stream({"input": document.page_content}):
    #     print(chunk.content, end="", flush=True)
    # Construct a graph document
    graph_document = GraphDocument(
        nodes=[map_to_base_node(node) for node in data.nodes],
        relationships=[map_to_base_relationship(rel) for rel in data.rels],
        source=document
    )
    # Store information into a graph
    graph.add_graph_documents([graph_document])

In [ ]:
from tqdm import tqdm

for i, d in tqdm(enumerate(split_docs), total=len(split_docs)):
    extract_and_store_graph(d)

  0%|          | 0/12 [01:36<?, ?it/s]


KeyboardInterrupt: 